In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv()

# MongoDB connection
MONGO_URI = os.getenv("MONGO_URI")
client = MongoClient(MONGO_URI)
db = client["TaskAssistant"]
tasks_collection = db["tasks"]

tasks_collection.insert_one({"name": "Hadil", "email": "example@gmail.com"})



In [ ]:
import subprocess
import sys
import os
import json

# Test if server starts
server_path = "backend/server.py"  # Adjust this to your actual path
print(f"Testing server at: {"backend\\server.py"}")

try:
    # Start the server process
    process = subprocess.Popen(
        [sys.executable, server_path],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    # Send an MCP initialize request
    init_request = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "initialize",
        "params": {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {
                "name": "test-client",
                "version": "1.0.0"
            }
        }
    }
    
    # Write request
    process.stdin.write(json.dumps(init_request) + "\n")
    process.stdin.flush()
    
    # Try to read response (with timeout)
    import select
    import time
    
    # Wait a bit for response
    time.sleep(2)
    
    # Check if process is still running
    if process.poll() is not None:
        # Process terminated
        stdout, stderr = process.communicate()
        print("❌ Server terminated immediately!")
        print(f"\nSTDOUT:\n{stdout}")
        print(f"\nSTDERR:\n{stderr}")
        print(f"\nReturn code: {process.returncode}")
    else:
        # Try to read output
        output = process.stdout.readline()
        print(f"✅ Server responded: {output}")
        process.terminate()
        
except Exception as e:
    print(f"❌ Error testing server: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# test_mongo.py
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
print(f"MongoDB URI: {MONGO_URI}")

try:
    client = MongoClient(MONGO_URI)
    db = client["TaskAssistant"]
    print("✅ MongoDB connection successful!")
    print(f"Collections: {db.list_collection_names()}")
except Exception as e:
    print(f"❌ MongoDB connection failed: {e}")

In [ ]:
import json
import os
from dotenv import load_dotenv
import requests

# Test 1: Check if .env file loads correctly
print("=" * 50)
print("TEST 1: Loading .env file")
print("=" * 50)

load_dotenv()

testmail_api_key = os.environ.get("TESTMAIL_API_KEY")
testmail_namespace = os.environ.get("TESTMAIL_NAMESPACE")

print(f"TESTMAIL_API_KEY found: {bool(testmail_api_key)}")
print(f"TESTMAIL_NAMESPACE found: {bool(testmail_namespace)}")

if testmail_api_key:
    print(f"API Key (masked): {testmail_api_key[:8]}...{testmail_api_key[-4:]}")
if testmail_namespace:
    print(f"Namespace: {testmail_namespace}")

print()

# Test 2: Verify API credentials
print("=" * 50)
print("TEST 2: Verifying Testmail API Connection")
print("=" * 50)

if testmail_api_key and testmail_namespace:
    try:
        test_url = f"https://api.testmail.app/api/json?apikey={testmail_api_key}&namespace={testmail_namespace}&livequery=true"
        response = requests.get(test_url, timeout=10)
        
        if response.status_code == 200:
            print("✅ SUCCESS: API credentials are valid!")
            print(f"Inbox URL: https://testmail.app/inbox/{testmail_namespace}")
        elif response.status_code == 401:
            print("❌ ERROR: Invalid API key or namespace")
        else:
            print(f"⚠️ WARNING: Unexpected status code: {response.status_code}")
            print(f"Response: {response.text}")
    except Exception as e:
        print(f"❌ ERROR: Connection failed - {str(e)}")
else:
    print("❌ ERROR: Missing API key or namespace in environment variables")

print()

# Test 3: Generate a test email address
print("=" * 50)
print("TEST 3: Generate Test Email Address")
print("=" * 50)

if testmail_namespace:
    test_email = f"test.{testmail_namespace}@inbox.testmail.app"
    print(f"Test email address: {test_email}")
    print(f"View inbox at: https://testmail.app/inbox/{testmail_namespace}/test")
else:
    print("❌ Cannot generate test email - namespace not configured")

print()

# Test 4: Send a test email
print("=" * 50)
print("TEST 4: Sending Test Email")
print("=" * 50)

if testmail_api_key and testmail_namespace:
    try:
        url = "https://api.testmail.app/api/send"
        
        headers = {
            "Authorization": f"Bearer {testmail_api_key}",
            "Content-Type": "application/json"
        }
        
        test_email_address = f"test.{testmail_namespace}@inbox.testmail.app"
        
        payload = {
            "from": f"noreply@{testmail_namespace}.testmail.app",
            "to": test_email_address,
            "subject": "Test Email - Configuration Check",
            "html": """
                <html>
                    <body style="font-family: Arial, sans-serif;">
                        <h2>✅ Email Configuration Successful!</h2>
                        <p>Your Testmail integration is working correctly.</p>
                        <p>You can now send emails from your chatbot.</p>
                    </body>
                </html>
            """,
            "text": "Email Configuration Successful! Your Testmail integration is working correctly."
        }
        
        response = requests.post(url, json=payload, headers=headers, timeout=30)
        
        if response.status_code == 200:
            result = response.json()
            print("✅ SUCCESS: Test email sent!")
            print(f"To: {test_email_address}")
            print(f"Message ID: {result.get('message_id', 'N/A')}")
            print(f"\nCheck your inbox at: https://testmail.app/inbox/{testmail_namespace}/test")
        else:
            print(f"❌ ERROR: Failed to send email")
            print(f"Status code: {response.status_code}")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
else:
    print("❌ Cannot send test email - missing credentials")

print()
print("=" * 50)
print("DIAGNOSIS COMPLETE")
print("=" * 50)